In [27]:
import sys
import os

# Add the src directory to PYTHONPATH
sys.path.append(os.path.abspath("src"))

In [28]:
%pwd

'd:\\MyProjects\\Project-01\\Text-Summarizer-Project'

In [29]:

# os.chdir('../')

# import os
os.chdir("d:/MyProjects/Project-01/Text-Summarizer-Project")
# print("Current Working Directory:", os.getcwd())

In [30]:
%pwd

'd:\\MyProjects\\Project-01\\Text-Summarizer-Project'

In [31]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path
    

In [32]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [33]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, 
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(str(config_filepath))
        self.params = read_yaml(str(params_filepath))
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion 
        create_directories([config.root_dir]) 

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config   


In [34]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size





In [35]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        try:
            if not os.path.exists(self.config.local_data_file):
                file_name, _ = request.urlretrieve(
                    url=self.config.source_url,
                    filename=self.config.local_data_file
                )
                logger.info(f"{file_name} downloaded successfully")
            else:
                logger.info(f"File already exists of size {get_size(Path(self.config.local_data_file))}")
        except request.HTTPError as http_err:
            logger.error(f"HTTP error occurred: {http_err.code} - {http_err.reason}")
        except Exception as e:
            logger.error(f"An unexpected error occurred: {e}")
            raise 
        

    def extract_zip_file(self):
        """
        Extracts the zip file to the specified directory.
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        try:
            logger.info(f"Extracting file: {self.config.local_data_file}")
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
                logger.info(f"Extracted {self.config.local_data_file} to {unzip_path}")
        except zipfile.BadZipFile as e:
            logger.error(f"Bad ZIP file: {e}")
            raise e
        except Exception as e:
            logger.error(f"An unexpected error occurred during extraction: {e}", exc_info=True)
            raise e

    # def extract_zip_file(self):
    #     """
    #     zip_file: str
    #     Extracts the zip file to the specified directory.
    #     Function returns None.
    #     """
    #     unzip_path = self.config.unzip_dir
    #     os.makedirs(unzip_path, exist_ok=True)
    #     with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
    #         zip_ref.extractall(unzip_path)
    #         logger.info(f"Extracted {self.config.local_data_file} to {unzip_path}") 


In [36]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except FileNotFoundError as fnf_error:
    logger.error(f"File not found: {fnf_error}")
except Exception as e:
    raise e    

[2025-04-12 18:14:32,281: INFO: common: 39: Directory: artifacts created successfully]
[2025-04-12 18:14:32,281: INFO: common: 39: Directory: artifacts/data_ingestion created successfully]
[2025-04-12 18:14:35,336: INFO: 2705554250: 12: artifacts/data_ingestion/summarizer-data.zip downloaded successfully]
[2025-04-12 18:14:35,336: INFO: 2705554250: 29: Extracting file: artifacts/data_ingestion/summarizer-data.zip]
[2025-04-12 18:14:35,534: INFO: 2705554250: 32: Extracted artifacts/data_ingestion/summarizer-data.zip to artifacts/data_ingestion]
